### Bioinformatics and Beyond Final Project
#### Goal: Classify mutations using K-Nearest-Neighbors 
Resources:
* A human gene
    * Specifically, [Homo sapiens hexosaminidase subunit alpha (HEXA)](https://www.ncbi.nlm.nih.gov/nuccore/NG_009017.1?from=4707&to=37745&report=fasta)
* K-Nearest-Neighbors
* TF-IDF (?)

In [1]:
import re
import json
import random
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
sequence = "".join(open("data/HEXA.fasta").read().split("\n")[1:])

In [3]:
class MutationsClassifier:
    def __init__(self, sequence, n=100):
        self.genetic_code_dictionary, self.amino_acid_dictionary = json.load(open("codons.json"))
        
        self.default_nucleotides = "ATCG"
        self.codons_pattern = re.compile(r".{3}")
        
        self.number_mutations = n
        
        self.original_sequence = sequence
        
    def point_mutation(self, sequence):
        mutation_index = random.randint(0, len(sequence) - 1)
        mutation_index_nucleotide = sequence[mutation_index]

        mutation_beginning = sequence[:mutation_index]
        mutation_middle = random.choice(self.default_nucleotides.replace(mutation_index_nucleotide, ""))
        mutation_end = sequence[mutation_index + 1:]

        mutation_final = mutation_beginning + mutation_middle + mutation_end

        if mutation_final == sequence:
            mutation_final = point_mutation(sequence)

        return mutation_final
    
    def insertion_mutation(self, sequence):
        mutation_index = random.randint(0, len(sequence) - 1)
        return sequence[:mutation_index] + random.choice(self.default_nucleotides) + sequence[mutation_index:]
    
    def deletion_mutation(self, sequence):
        mutation_index = random.randint(0, len(sequence) - 1)
        return sequence[:mutation_index] + sequence[mutation_index + 1:]
    
    def copy_number_addition_mutation(self, sequence):
        for i in range(3, (len(sequence)//2) + 1):
            kmers = [sequence[j:j + i] for j in range(len(sequence) - i + 1)]
            for k, el in enumerate(kmers):
                if k + i < len(kmers) and el == kmers[k + i]:
                    match_index = k + (i * 2)
                    return sequence[:match_index] + el + sequence[match_index:]
    
    def copy_number_subtraction_mutation(self, sequence):
        for i in range(3, (len(sequence)//2) + 1):
            kmers = [sequence[j:j + i] for j in range(len(sequence) - i + 1)]
            for k, el in enumerate(kmers):
                if k + i < len(kmers) and el == kmers[k + i]:
                    match_index = k + (i * 2)
                    return sequence[:match_index - i] + sequence[match_index:]
                
    def classify(self, sequences):
        self.sequence_and_mutations_dictionary = {"original_sequence": self.original_sequence, 
                                                  "mutations": 
                                                  {"point_mutations":
                                                   [(" ".join(self.codons_pattern
                                                    .findall(self.point_mutation
                                                             (self.original_sequence)))) 
                                                    for _ in range(self.number_mutations)], 
                                                   "insertion_mutations": 
                                                   [(" ".join(self.codons_pattern
                                                    .findall(self.insertion_mutation
                                                             (self.original_sequence)))) 
                                                    for _ in range(self.number_mutations)], 
                                                   "deletion_mutations": 
                                                   [(" ".join(self.codons_pattern
                                                    .findall(self.deletion_mutation
                                                             (self.original_sequence)))) 
                                                    for _ in range(self.number_mutations)], 
                                                   "copy_number_addition_mutations": 
                                                   [(" ".join(self.codons_pattern
                                                    .findall(self.copy_number_addition_mutation
                                                             (self.original_sequence)))) 
                                                    for _ in range(self.number_mutations)], 
                                                   "copy_number_subtraction_mutations": 
                                                   [(" ".join(self.codons_pattern
                                                    .findall(self.copy_number_subtraction_mutation
                                                             (self.original_sequence)))) 
                                                    for _ in range(self.number_mutations)]}}
            
        self.mutations_and_types_list = []
        for mutation_type, list_of_mutations in self.sequence_and_mutations_dictionary["mutations"].items():
            for mutation in list_of_mutations:
                self.mutations_and_types_list.append((mutation, mutation_type))
        
        self.pipeline = Pipeline([
            ("vectorizer", TfidfVectorizer()),
            ("classifier", KNeighborsClassifier(10))
        ])
        
        self.pipeline.fit([pair[0] for pair in self.mutations_and_types_list], 
                          [pair[1] for pair in self.mutations_and_types_list])
        
        return self.pipeline.predict(sequences)

In [4]:
%%time
MutationsClassifier(sequence).classify(["AAA TTT AAA TTT AAA TTT AAA TTT AAA", 
                                        "AAA AAA AAA AAA AAA AAA AAA AAA AAA", 
                                        "TTT TTT TTT TTT TTT TTT TTT TTT TTT", 
                                        "GGG CCC GGG CCC GGG CCC GGG CCC GGG", 
                                        "CGC GCG CGC GCG CGC GCG CGC GCG CGC"]).tolist()

CPU times: user 5.56 s, sys: 117 ms, total: 5.68 s
Wall time: 5.73 s


['insertion_mutations',
 'insertion_mutations',
 'deletion_mutations',
 'deletion_mutations',
 'insertion_mutations']